In [1]:
from __future__ import print_function

import os
import sys
sys.path.insert(0, os.path.abspath("../.."))

# IBM PAIRS open-source module
from ibmpairs import paw
import covid19_userLocal as covid19

import numpy
import pandas
import geopandas
from shapely import wkb, wkt

from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta
import pytz

In [2]:
# For Both Cases and Growth we use the same ROLLING_WINDOW 
ROLLING_WINDOW = 14
lag_growthCases = 19
lag_growthCasesStd = 4 

lag_Feature = numpy.arange(lag_growthCases-lag_growthCasesStd, lag_growthCases+lag_growthCasesStd+1, 1)
print('lag_Feature', lag_Feature)

#dt_cutoff_training_COVID = datetime(2020,5,31, tzinfo=pytz.utc)
dt_cutoff_training_COVID = datetime(2020,7,25, tzinfo=pytz.utc)
print('Training only with COVID growth data until ', dt_cutoff_training_COVID)
dt_cutoff_training_mobility = dt_cutoff_training_COVID - timedelta(days=lag_growthCases - lag_growthCasesStd)
print('Training only with Mobility data until     ', dt_cutoff_training_mobility)
dt_cutoff_min = datetime(2020, 3, 1, tzinfo=pytz.utc)
print('Considering data from                      ', dt_cutoff_min)
dt_cutoff_latest = datetime(2020,8,2, tzinfo=pytz.utc)
print('Plotting data up to                        ', dt_cutoff_latest)

data_subdirectory = 'data/csv/run011FullTrainingJHU'
if not os.path.exists(data_subdirectory):
    os.makedirs(data_subdirectory)
print('data_subdirectory                          ', data_subdirectory)

lag_Feature [15 16 17 18 19 20 21 22 23]
Training only with COVID growth data until  2020-07-25 00:00:00+00:00
Training only with Mobility data until      2020-07-10 00:00:00+00:00
Considering data from                       2020-03-01 00:00:00+00:00
Plotting data up to                         2020-08-02 00:00:00+00:00
data_subdirectory                           data/csv/run011FullTrainingJHU


In [3]:
# One-time calculation
"""
# Local Polygons
df_region = pandas.read_csv('data/local_polygons.csv', usecols=['id', 'name', 'poly'])

df_region['poly'] = df_region['poly'].apply(lambda x: wkb.loads(x, hex=True))
df_region = df_region.rename(columns={'id': 'pairs_id'})
df_region = geopandas.GeoDataFrame(df_region, geometry='poly')

# We need County and State columns later on
new = df_region['name'].str.split('.', expand=True)
df_region['County'] = new[1]
df_region['State'] = new[0]

# Write it out without the wkb.loads
df_region_csv = df_region.copy()
del df_region_csv['poly']
df_tmp = pandas.read_csv('data/local_polygons.csv')[['id', 'poly']]
df_tmp = df_tmp.rename(columns={'id': 'pairs_id'})
df_region_csv = pandas.merge(df_region_csv, df_tmp, on='pairs_id', how='left')
df_region_csv.to_csv('data/df_region.csv', index=None)

df_region.tail()
"""

"\n# Local Polygons\ndf_region = pandas.read_csv('data/local_polygons.csv', usecols=['id', 'name', 'poly'])\n\ndf_region['poly'] = df_region['poly'].apply(lambda x: wkb.loads(x, hex=True))\ndf_region = df_region.rename(columns={'id': 'pairs_id'})\ndf_region = geopandas.GeoDataFrame(df_region, geometry='poly')\n\n# We need County and State columns later on\nnew = df_region['name'].str.split('.', expand=True)\ndf_region['County'] = new[1]\ndf_region['State'] = new[0]\n\n# Write it out without the wkb.loads\ndf_region_csv = df_region.copy()\ndel df_region_csv['poly']\ndf_tmp = pandas.read_csv('data/local_polygons.csv')[['id', 'poly']]\ndf_tmp = df_tmp.rename(columns={'id': 'pairs_id'})\ndf_region_csv = pandas.merge(df_region_csv, df_tmp, on='pairs_id', how='left')\ndf_region_csv.to_csv('data/df_region.csv', index=None)\n\ndf_region.tail()\n"

In [4]:
# Get the region data (county ids, names, and polygons)
df_region = pandas.read_csv('data/df_region.csv')
# Make a copy in the specific subfolder
df_region.to_csv(os.path.join(data_subdirectory, 'df_region.csv'), index=None)
# Read back
df_region = pandas.read_csv(os.path.join(data_subdirectory, 'df_region.csv'))
df_region['poly'] = df_region['poly'].apply(lambda x: wkb.loads(x, hex=True))
df_region = geopandas.GeoDataFrame(df_region, geometry='poly')

df_region.tail()

,pairs_id,name,County,State,poly
3136,53137,Wyoming.Niobrara,Niobrara,Wyoming,"POLYGON ((-104.05916 43.47913, -104.05620 43.0..."
3137,53138,Wyoming.Converse,Converse,Wyoming,"POLYGON ((-104.90069 42.61193, -105.28875 42.6..."
3138,53139,Wyoming.Goshen,Goshen,Wyoming,"POLYGON ((-104.05351 41.99982, -104.05362 41.6..."
3139,53140,Wyoming.Platte,Platte,Wyoming,"POLYGON ((-104.65064 41.65637, -105.28110 41.6..."
3140,53141,Wyoming.Laramie,Laramie,Wyoming,"POLYGON ((-104.05550 41.56422, -104.05401 41.3..."


In [5]:
# Query Local COVID-19 Cases
coronaQueryLocal = covid19.query_local(layerID='P567C6007') #JHU
df_local_covid = coronaQueryLocal.vdf[['timestamp', 'pairs_id', 'State', 'County', 'Value']]
df_local_covid = df_local_covid.rename(columns={'Value': 'Cases'})
df_local_covid['pairs_id'] = df_local_covid['pairs_id'].astype(int)
df_local_covid = df_local_covid[df_local_covid['timestamp']<=dt_cutoff_latest].reset_index(drop=True)

df_local_covid = pandas.merge(df_local_covid, df_region[['pairs_id']], on='pairs_id').reset_index()

# Make a copy in the specific subfolder
df_local_covid.to_csv(os.path.join(data_subdirectory, 'df_local_covid.csv'), index=None)

df_local_covid.tail()

Be aware: As directed, downloading query result into memory!
No PAIRS query ZIP file identified, or no acknowledgement in ZIP file found. Did you run `self.download()`, yet?
No PAIRS meta data file 'output.info' found


,index,timestamp,pairs_id,State,County,Cases
604654,604654,2020-07-29 00:00:00+00:00,53058,Wisconsin,Price,18
604655,604655,2020-07-30 00:00:00+00:00,53058,Wisconsin,Price,18
604656,604656,2020-07-31 00:00:00+00:00,53058,Wisconsin,Price,19
604657,604657,2020-08-01 00:00:00+00:00,53058,Wisconsin,Price,21
604658,604658,2020-08-02 00:00:00+00:00,53058,Wisconsin,Price,22


In [6]:
# Read back covid data from csv (raw cumulative cases)
df_local_covid = pandas.read_csv(os.path.join(data_subdirectory, 'df_local_covid.csv'))
df_local_covid['timestamp'] = pandas.to_datetime(df_local_covid['timestamp'])
df_local_covid.tail()

,index,timestamp,pairs_id,State,County,Cases
604654,604654,2020-07-29 00:00:00+00:00,53058,Wisconsin,Price,18
604655,604655,2020-07-30 00:00:00+00:00,53058,Wisconsin,Price,18
604656,604656,2020-07-31 00:00:00+00:00,53058,Wisconsin,Price,19
604657,604657,2020-08-01 00:00:00+00:00,53058,Wisconsin,Price,21
604658,604658,2020-08-02 00:00:00+00:00,53058,Wisconsin,Price,22


In [7]:
# Query Local Mobility (Descartes lab median of max mobility)
mobilityQueryLocal = covid19.query_local(layerID='P612C6303')
df_local_mobility = mobilityQueryLocal.vdf[['timestamp', 'pairs_id', 'State', 'County', 'Value']]
df_local_mobility = df_local_mobility.rename(columns={'Value': 'Mobility'})
df_local_mobility['pairs_id'] = df_local_mobility['pairs_id'].astype(int)
df_local_mobility = df_local_mobility[df_local_mobility['timestamp']<=dt_cutoff_latest].reset_index(drop=True)

# Make a copy in the specific subfolder
df_local_mobility.to_csv(os.path.join(data_subdirectory, 'df_local_mobility.csv'), index=None)

df_local_mobility.tail()

Be aware: As directed, downloading query result into memory!
No PAIRS query ZIP file identified, or no acknowledgement in ZIP file found. Did you run `self.download()`, yet?
No PAIRS meta data file 'output.info' found


,timestamp,pairs_id,State,County,Mobility
404375,2020-08-02 00:00:00+00:00,51963,North_Carolina,Dare,5.070
404376,2020-08-02 00:00:00+00:00,50320,Florida,Jackson,8.357
404377,2020-08-02 00:00:00+00:00,50515,Georgia,Ben_Hill,2.422
404378,2020-08-02 00:00:00+00:00,50160,Arkansas,Phillips,3.233
404379,2020-08-02 00:00:00+00:00,52918,Virginia,Carroll,10.454


In [8]:
# Read back mobility data (Descartes lab median of max mobility)
df_local_mobility = pandas.read_csv(os.path.join(data_subdirectory, 'df_local_mobility.csv'))
df_local_mobility['timestamp'] = pandas.to_datetime(df_local_mobility['timestamp'])
df_local_mobility.tail()

,timestamp,pairs_id,State,County,Mobility
404375,2020-08-02 00:00:00+00:00,51963,North_Carolina,Dare,5.070
404376,2020-08-02 00:00:00+00:00,50320,Florida,Jackson,8.357
404377,2020-08-02 00:00:00+00:00,50515,Georgia,Ben_Hill,2.422
404378,2020-08-02 00:00:00+00:00,50160,Arkansas,Phillips,3.233
404379,2020-08-02 00:00:00+00:00,52918,Virginia,Carroll,10.454


In [9]:
# Unstacking COVID19 and mobility

# Unstack the COVID19 data and first derivative (new cases)
df_unstacked = df_local_covid.copy()
del df_unstacked['State']
del df_unstacked['County']

df_unstacked = df_unstacked.set_index(['timestamp', 'pairs_id']).unstack().reset_index().sort_values(by='timestamp').set_index('timestamp')
df_unstacked = df_unstacked.swaplevel(axis=1)
df_unstacked = df_unstacked.replace(0, numpy.nan)

# Replace values where no change with nan so that the daily numbers make sense when reporting only every couple of days
# (also replace values with negative change)
df_unstacked[df_unstacked.diff()<=0] = numpy.nan

# Interpolate
df_unstacked = df_unstacked.interpolate(method='linear', limit_area='inside')

# New Local Cases (1st derivative)
df_new = df_unstacked.diff()


# Unstack the mobility data
df_m_unstacked = df_local_mobility.copy()
del df_m_unstacked['State']
del df_m_unstacked['County']

df_m_unstacked = df_m_unstacked.set_index(['timestamp', 'pairs_id']).unstack().reset_index().sort_values(by='timestamp').set_index('timestamp')
df_m_unstacked = df_m_unstacked.swaplevel(axis=1)

# Erase high-value mobility outliers >100miles before taking the rolling mean
df_m_unstacked = df_m_unstacked.clip(upper=100)

# Interpolate
df_m_unstacked = df_m_unstacked.interpolate(method='linear', limit_area='inside')

df_m_unstacked.tail()


# Stack and merge in order to fill in nan at all missing combinations
df_stacked = pandas.merge(df_new.stack(level='pairs_id').reset_index(),
                          df_m_unstacked.stack(level='pairs_id').reset_index(),
                          on=['timestamp', 'pairs_id'],
                          how='outer'
                         )
df_stacked['pairs_id'] = df_stacked['pairs_id'].astype(int)

# Unstack again
df_stacked = df_stacked.set_index(['timestamp', 'pairs_id']).unstack().reset_index().sort_values(
    by='timestamp').set_index('timestamp')

df_new = df_stacked[['Cases']].swaplevel(axis=1)
df_m_unstacked = df_stacked[['Mobility']].swaplevel(axis=1)

df_new.tail()

pairs_id,50001,50002,50003,50004,50005,50006,50007,50008,50009,50010,...,53132,53133,53134,53135,53136,53137,53138,53139,53140,53141
,Cases,Cases,Cases,Cases,Cases,Cases,Cases,Cases,Cases,Cases,...,Cases,Cases,Cases,Cases,Cases,Cases,Cases,Cases,Cases,Cases
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-07-29 00:00:00+00:00,NaN,41.0,142.0,60.0,15.0,72.0,27.0,16.0,12.0,28.0,...,2.0,1.0,1.0,2.0,6.0,NaN,0.083333,2.00,NaN,10.0
2020-07-30 00:00:00+00:00,NaN,34.0,79.0,32.0,14.0,36.0,18.0,14.0,7.0,32.0,...,2.0,4.0,1.0,3.0,3.0,NaN,0.083333,0.25,NaN,7.0
2020-07-31 00:00:00+00:00,NaN,13.0,98.0,9.0,3.0,22.0,5.0,5.0,8.0,35.0,...,7.0,1.0,4.0,1.0,3.0,NaN,0.083333,0.25,NaN,7.0
2020-08-01 00:00:00+00:00,NaN,28.0,118.0,26.0,8.0,45.0,21.0,8.0,3.0,32.0,...,5.0,0.5,3.0,1.0,2.0,NaN,0.083333,0.25,NaN,6.0
2020-08-02 00:00:00+00:00,NaN,13.0,73.0,18.0,5.0,16.0,13.0,6.0,2.0,13.0,...,3.0,0.5,NaN,2.0,5.0,NaN,0.083333,0.25,NaN,11.0


In [10]:
# FIPS codes to pairs_id
df_fips = pandas.read_csv('data/County_PAIRS_FIPS.csv', dtype = {'FIPS' : 'string'})

# Make a copy in the specific subfolder
df_fips.to_csv(os.path.join(data_subdirectory, 'County_PAIRS_FIPS.csv'), index=None)

# Read back translation FIPS codes to pairs_id
df_fips = pandas.read_csv(os.path.join(data_subdirectory, 'County_PAIRS_FIPS.csv'), dtype = {'FIPS' : 'string'})
df_fips.tail()

,pairs_id,FIPS
3137,53137,56027
3138,53138,56009
3139,53139,56015
3140,53140,56031
3141,53141,56021


In [11]:
# Census data
df_census = pandas.read_csv(
    'data/cc-est2019-alldata.csv',
    dtype = {'STATE' : 'string', 'COUNTY' : 'string'},
    encoding='latin-1'
)
df_census['FIPS'] = df_census['STATE'] + df_census['COUNTY']

# Use only latest (2019 estimate)
df_census = df_census[df_census['YEAR']==12]
df_census.tail()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE,FIPS
716371,50,56,045,Wyoming,Weston County,12,14,499,280,219,...,5,0,0,0,0,0,0,0,0,56045
716372,50,56,045,Wyoming,Weston County,12,15,352,180,172,...,2,0,0,0,0,3,0,0,0,56045
716373,50,56,045,Wyoming,Weston County,12,16,229,107,122,...,0,0,0,0,0,0,0,0,0,56045
716374,50,56,045,Wyoming,Weston County,12,17,198,82,116,...,1,0,0,1,0,0,0,0,0,56045
716375,50,56,045,Wyoming,Weston County,12,18,200,72,128,...,2,0,0,0,0,0,0,0,0,56045


In [12]:
# Absolute population numbers

df_population = df_census[df_census['AGEGRP']==0][['FIPS', 'TOT_POP']].reset_index(drop=True)  #Total population
df_population = pandas.merge(df_fips[['pairs_id', 'FIPS']], df_population, on='FIPS').drop(columns='FIPS')
df_population = df_population.rename(columns={'TOT_POP': 'population'})
# Even after removing duplicates there are two FIPS in Alaska pointing to the same pairs_id so groupby sum pairs_id
df_population = df_population.groupby('pairs_id').sum().reset_index().sort_values(by='pairs_id')
df_population.tail()

# Make a copy in the specific subfolder
df_population.to_csv(os.path.join(data_subdirectory, 'df_population.csv'), index=None)

# Read back from disk
df_population = pandas.read_csv(os.path.join(data_subdirectory, 'df_population.csv'))

df_population.tail()

,pairs_id,population
3135,53137,2356
3136,53138,13822
3137,53139,13211
3138,53140,8393
3139,53141,99500


In [13]:
# Calculate the population density (one-time calculation)
"""
import shapely.ops as ops
import pyproj
from functools import partial

def geom_area(geom):
    # Calculate area for lat-lon polygon in km2
    geom_transformed = ops.transform(
        partial(
            pyproj.transform,
            pyproj.Proj(init='EPSG:4326'),
            pyproj.Proj(
                proj='aea',
                lat_1=geom.bounds[1],
                lat_2=geom.bounds[3])),
        geom)
    return geom_transformed.area / 1e6

df_pop_density = pandas.merge(df_population, df_region[['pairs_id', 'poly']], on='pairs_id')
#df_pop_density['population_density'] = df_pop_density['population'] / df_pop_density['poly'].apply(lambda x: x.area)
df_pop_density['population_density'] = df_pop_density['population'] / df_pop_density['poly'].apply(lambda x: geom_area(x))
del df_pop_density['poly']
del df_pop_density['population']

# Write to disk
df_pop_density.to_csv('data/df_pop_density.csv', index=False)
"""

"\nimport shapely.ops as ops\nimport pyproj\nfrom functools import partial\n\ndef geom_area(geom):\n    # Calculate area for lat-lon polygon in km2\n    geom_transformed = ops.transform(\n        partial(\n            pyproj.transform,\n            pyproj.Proj(init='EPSG:4326'),\n            pyproj.Proj(\n                proj='aea',\n                lat_1=geom.bounds[1],\n                lat_2=geom.bounds[3])),\n        geom)\n    return geom_transformed.area / 1e6\n\ndf_pop_density = pandas.merge(df_population, df_region[['pairs_id', 'poly']], on='pairs_id')\n#df_pop_density['population_density'] = df_pop_density['population'] / df_pop_density['poly'].apply(lambda x: x.area)\ndf_pop_density['population_density'] = df_pop_density['population'] / df_pop_density['poly'].apply(lambda x: geom_area(x))\ndel df_pop_density['poly']\ndel df_pop_density['population']\n\n# Write to disk\ndf_pop_density.to_csv('data/df_pop_density.csv', index=False)\n"

In [14]:
# Read population density from disk
df_pop_density = pandas.read_csv('data/df_pop_density.csv')

# Make a copy in the specific subfolder
df_pop_density.to_csv(os.path.join(data_subdirectory, 'df_pop_density.csv'), index=None)

# Read back from disk
df_pop_density = pandas.read_csv(os.path.join(data_subdirectory, 'df_pop_density.csv'))
df_pop_density.tail()

,pairs_id,population_density
3135,53137,0.346710
3136,53138,1.254922
3137,53139,2.286790
3138,53140,1.519373
3139,53141,14.217309


In [15]:
# Age-related census population data

#AGEGRP
#0 = Total
#1 = Age 0 to 4 years
#2 = Age 5 to 9 years
#3 = Age 10 to 14 years
#4 = Age 15 to 19 years
#5 = Age 20 to 24 years
#6 = Age 25 to 29 years
#7 = Age 30 to 34 years
#8 = Age 35 to 39 years
#9 = Age 40 to 44 years
#10 = Age 45 to 49 years
#11 = Age 50 to 54 years
#12 = Age 55 to 59 years
#13 = Age 60 to 64 years
#14 = Age 65 to 69 years
#15 = Age 70 to 74 years
#16 = Age 75 to 79 years
#17 = Age 80 to 84 years
#18 = Age 85 years or older

df_age_20_24 = df_census[df_census['AGEGRP']==5].reset_index(drop=True)  #Age 20 to 24 years
df_age_60_64 = df_census[df_census['AGEGRP']==13].reset_index(drop=True)  #Age 60 to 64 years
df_AgeRatio = df_age_60_64[['FIPS', 'TOT_POP']].set_index(
    'FIPS') / df_age_20_24[['FIPS', 'TOT_POP']].set_index('FIPS')
df_AgeRatio = df_AgeRatio.rename(columns={'TOT_POP': 'AgeRatio'}).reset_index()

# Clip outliers
df_AgeRatio['AgeRatio'] = df_AgeRatio['AgeRatio'].clip(lower=0.1, upper=20)
df_AgeRatio['LogAgeRatio'] = numpy.log(df_AgeRatio['AgeRatio'])

df_AgeRatio = pandas.merge(df_fips[['pairs_id', 'FIPS']], df_AgeRatio, on='FIPS').drop(columns='FIPS')
# Even after removing duplicates there are two FIPS in Alaska pointing to the same pairs_id so groupby mean pairs_id
df_AgeRatio = df_AgeRatio.groupby('pairs_id').mean().reset_index().sort_values(by='pairs_id')

# Make a copy in the specific subfolder
df_AgeRatio.to_csv(os.path.join(data_subdirectory, 'df_AgeRatio.csv'), index=None)

# Read back from disk
df_AgeRatio = pandas.read_csv(os.path.join(data_subdirectory, 'df_AgeRatio.csv'))

df_AgeRatio.tail()

,pairs_id,AgeRatio,LogAgeRatio
3135,53137,1.466165,0.382650
3136,53138,1.474648,0.388419
3137,53139,1.263804,0.234126
3138,53140,1.867769,0.624744
3139,53141,0.951245,-0.049983


In [16]:
def process_covid_data(df_new, df_population, rolling_window_log, rolling_window_growth):
    """
    :df_new:    DataFrame with the new cases and fatalities
    
    Returns: df_log_new (Semi-log cleaned up daily cases and fatalities)
    Returns: df_log_new_rolling (Semi-log cleaned up daily data with 5 day rolling mean)
    Returns: df_growth (Growth in daily cases and fatalities)
    Returns: df_new_rolling_scaled (Daily cases and fatalities cleand up, rolling mean, normalized by 100K population)
    """
    # Build a semi-log version of the data and clean up
    df_log_new = df_new.copy()
    df_log_new[df_log_new<=0] = numpy.nan
    df_log_new = df_log_new.apply(lambda x: numpy.log(x))
    df_log_new = df_log_new.replace([numpy.inf, -numpy.inf], numpy.nan)
    
    # Remove outliers (non-symmetrical so that we don't erase too many valid high-value outliers)
    df_log_new[(df_log_new < df_log_new.rolling(3, center=True).mean() - 0.8) | 
               ((df_log_new > df_log_new.rolling(3, center=True).mean() + 1.2) & (df_log_new > 4))] = numpy.nan
    
    # Interpolate to fill in missing values
    df_log_new = df_log_new.interpolate(method='linear', limit_area='inside')
    
    # Rolling Mean
    df_log_new_rolling = df_log_new.rolling(rolling_window_log, min_periods=1).mean()
    
    # Growth rate (don't use min_periods=1 because it generates too many outliers)
    df_growth = df_log_new_rolling.diff()

    # Mask bad growth values due to derivatives of small numbers
    SMALL_VALUE = -1
    df_growth[df_log_new<=SMALL_VALUE] = numpy.nan

    # Interpolate inside to fill in missing values
    df_growth = df_growth.interpolate(method='linear', limit_area='inside')
    
    # Filling outside nans with zero
    df_growth = df_growth.fillna(0)
    
    # Apply rolling mean for growth
    df_growth = df_growth.rolling(rolling_window_growth, min_periods=1).mean()
    
    # Scale by 100K population
    df_new_rolling_scaled = numpy.exp(df_log_new_rolling)
    for pairs_id in df_log_new_rolling.columns.get_level_values('pairs_id').unique():
        df_new_rolling_scaled[pairs_id] = df_new_rolling_scaled[pairs_id] \
            * 100000. / df_population[df_population['pairs_id']==pairs_id]['population'].values[0]

    return df_log_new, df_log_new_rolling, df_growth, df_new_rolling_scaled

In [17]:
# Process the Covid data 
df_log_new, df_log_new_rolling, df_growth, df_new_rolling_scaled\
    = process_covid_data(df_new.swaplevel(axis=1)[['Cases']].swaplevel(axis=1),
                         df_population,
                         rolling_window_log=ROLLING_WINDOW, 
                         rolling_window_growth=ROLLING_WINDOW)

df_log_new_rolling.tail()

pairs_id,50001,50002,50003,50004,50005,50006,50007,50008,50009,50010,...,53132,53133,53134,53135,53136,53137,53138,53139,53140,53141
,Cases,Cases,Cases,Cases,Cases,Cases,Cases,Cases,Cases,Cases,...,Cases,Cases,Cases,Cases,Cases,Cases,Cases,Cases,Cases,Cases
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-07-29 00:00:00+00:00,NaN,3.287400,4.717454,3.279579,2.247524,3.727344,2.473464,2.320719,1.698845,2.590759,...,0.449398,0.779226,0.520930,0.666531,1.700903,NaN,-2.071996,-1.922198,-3.044522,1.761523
2020-07-30 00:00:00+00:00,NaN,3.283317,4.687001,3.291714,2.271558,3.705322,2.465938,2.306850,1.738817,2.611308,...,0.391474,0.878247,0.619951,0.745003,1.779375,NaN,-2.121507,-1.922198,-3.044522,1.851006
2020-07-31 00:00:00+00:00,NaN,3.212574,4.628916,3.203374,2.251572,3.638586,2.489394,2.211493,1.730404,2.708316,...,0.480957,0.878247,0.718972,0.745003,1.758826,NaN,-2.171017,-1.922198,-3.044522,1.841468
2020-08-01 00:00:00+00:00,NaN,3.203037,4.615691,3.198081,2.231585,3.637016,2.549916,2.149708,1.730404,2.757827,...,0.546407,0.878247,0.797444,0.666531,1.651392,NaN,-2.249490,-1.849941,-3.044522,1.854491
2020-08-02 00:00:00+00:00,NaN,3.134364,4.609151,3.174616,2.198014,3.577135,2.605143,2.106412,1.680894,2.776567,...,0.674389,0.878247,0.858786,0.602539,1.613614,NaN,-2.327962,-1.777683,-3.044522,1.886055


In [18]:
# Mobility rolling mean
df_Mobility_rolling = df_m_unstacked.rolling(ROLLING_WINDOW, min_periods=1).mean()

# LogMobility rolling mean
df_LogMobility_rolling = df_Mobility_rolling.swaplevel(axis=1).rename(
    columns={'Mobility': 'LogMobility'}).swaplevel(axis=1).apply(lambda x: numpy.log(x))

df_LogMobility_rolling.tail()

pairs_id,50001,50002,50003,50004,50005,50006,50007,50008,50009,50010,...,53132,53133,53134,53135,53136,53137,53138,53139,53140,53141
,LogMobility,LogMobility,LogMobility,LogMobility,LogMobility,LogMobility,LogMobility,LogMobility,LogMobility,LogMobility,...,LogMobility,LogMobility,LogMobility,LogMobility,LogMobility,LogMobility,LogMobility,LogMobility,LogMobility,LogMobility
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-07-29 00:00:00+00:00,2.183222,2.348180,2.000050,2.041461,2.665674,2.191119,2.389510,2.414018,2.342273,2.289189,...,2.273149,1.419574,4.055298,1.374276,1.506519,NaN,1.663453,0.996711,1.908939,1.511479
2020-07-30 00:00:00+00:00,2.179497,2.337676,1.989673,2.041934,2.672926,2.185971,2.385540,2.406701,2.321722,2.288812,...,2.262217,1.416304,4.050699,1.368803,1.501248,NaN,1.645563,0.962192,1.920530,1.509807
2020-07-31 00:00:00+00:00,2.173395,2.342321,1.979078,2.042666,2.679068,2.167828,2.401823,2.414797,2.318360,2.290035,...,2.265677,1.419815,4.068384,1.388898,1.504141,NaN,1.680921,0.893373,1.930113,1.511778
2020-08-01 00:00:00+00:00,2.171760,2.336848,1.974736,2.039029,2.681354,2.163971,2.407035,2.422087,2.310390,2.276967,...,2.246249,1.416945,4.025843,1.406185,1.498826,NaN,1.556835,0.859962,1.923372,1.510517
2020-08-02 00:00:00+00:00,2.165677,2.332102,1.967342,2.043574,2.686330,2.163799,2.399472,2.415231,2.322423,2.267269,...,2.214729,1.401218,4.011072,1.409627,1.480370,NaN,1.540073,0.839410,1.880359,1.496925


In [19]:
# Filter out data before dt_cutoff_min. Corona numbers are too low. 
# We are doing this here AFTER the rolling means have been applied
del df_new
del df_log_new
df_LogCases = df_log_new_rolling[df_log_new_rolling.index>=dt_cutoff_min].swaplevel(axis=1)['Cases']
del df_log_new_rolling
df_GrowthCases = df_growth[df_growth.index>=dt_cutoff_min].swaplevel(axis=1)['Cases']
del df_growth
df_CasesCapita = df_new_rolling_scaled[df_new_rolling_scaled.index>=dt_cutoff_min].swaplevel(axis=1)['Cases']
del df_new_rolling_scaled
df_Mobility = df_Mobility_rolling[df_Mobility_rolling.index>=dt_cutoff_min].swaplevel(axis=1)['Mobility']
del df_Mobility_rolling
df_LogMobility = df_LogMobility_rolling[df_LogMobility_rolling.index>=dt_cutoff_min].swaplevel(axis=1)['LogMobility']
del df_LogMobility_rolling

In [20]:
# Make a copy in the specific subfolder
df_LogCases.to_csv(os.path.join(data_subdirectory, 'df_LogCases.csv'))

# Make a copy in the specific subfolder
df_GrowthCases.to_csv(os.path.join(data_subdirectory, 'df_GrowthCases.csv'))

# Make a copy in the specific subfolder
df_CasesCapita.to_csv(os.path.join(data_subdirectory, 'df_CasesCapita.csv'))

# Make a copy in the specific subfolder
df_Mobility.to_csv(os.path.join(data_subdirectory, 'df_Mobility.csv'))

# Make a copy in the specific subfolder
df_LogMobility.to_csv(os.path.join(data_subdirectory, 'df_LogMobility.csv'))

In [21]:
# Get processed LogCases DataFrame
df_LogCases = pandas.read_csv(os.path.join(data_subdirectory, 'df_LogCases.csv'), index_col='timestamp')
df_LogCases.index = pandas.to_datetime(df_LogCases.index)
df_LogCases.columns = df_LogCases.columns.astype(int)
df_LogCases.columns.name = 'pairs_id'
df_LogCases.tail()

pairs_id,50001,50002,50003,50004,50005,50006,50007,50008,50009,50010,...,53132,53133,53134,53135,53136,53137,53138,53139,53140,53141
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-07-29 00:00:00+00:00,NaN,3.287400,4.717454,3.279579,2.247524,3.727344,2.473464,2.320719,1.698845,2.590759,...,0.449398,0.779226,0.520930,0.666531,1.700903,NaN,-2.071996,-1.922198,-3.044522,1.761523
2020-07-30 00:00:00+00:00,NaN,3.283317,4.687001,3.291714,2.271558,3.705322,2.465938,2.306850,1.738817,2.611308,...,0.391474,0.878247,0.619951,0.745003,1.779375,NaN,-2.121507,-1.922198,-3.044522,1.851006
2020-07-31 00:00:00+00:00,NaN,3.212574,4.628916,3.203374,2.251572,3.638586,2.489394,2.211493,1.730404,2.708316,...,0.480957,0.878247,0.718972,0.745003,1.758826,NaN,-2.171017,-1.922198,-3.044522,1.841468
2020-08-01 00:00:00+00:00,NaN,3.203037,4.615691,3.198081,2.231585,3.637016,2.549916,2.149708,1.730404,2.757827,...,0.546407,0.878247,0.797444,0.666531,1.651392,NaN,-2.249490,-1.849941,-3.044522,1.854491
2020-08-02 00:00:00+00:00,NaN,3.134364,4.609151,3.174616,2.198014,3.577135,2.605143,2.106412,1.680894,2.776567,...,0.674389,0.878247,0.858786,0.602539,1.613614,NaN,-2.327962,-1.777683,-3.044522,1.886055


In [22]:
# Get processed GrowthCases DataFrame
df_GrowthCases = pandas.read_csv(os.path.join(data_subdirectory, 'df_GrowthCases.csv'), index_col='timestamp')
df_GrowthCases.index = pandas.to_datetime(df_GrowthCases.index)
df_GrowthCases.columns = df_GrowthCases.columns.astype(int)
df_GrowthCases.columns.name = 'pairs_id'
df_GrowthCases.tail()

pairs_id,50001,50002,50003,50004,50005,50006,50007,50008,50009,50010,...,53132,53133,53134,53135,53136,53137,53138,53139,53140,53141
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-07-29 00:00:00+00:00,0.0,0.009676,-0.003423,0.036703,0.055471,0.018258,0.038096,0.014277,-0.000102,-0.012423,...,-0.017746,-0.015209,0.115924,0.045604,0.024454,0.0,3.965082e-18,0.106433,0.022505,0.013854
2020-07-30 00:00:00+00:00,0.0,0.009385,-0.005351,0.029896,0.052512,0.014009,0.034952,0.010041,0.000355,-0.012714,...,-0.023952,0.000075,0.116213,0.044136,0.030059,0.0,3.965082e-18,0.094415,0.016879,0.029387
2020-07-31 00:00:00+00:00,0.0,-0.000343,-0.015082,0.016675,0.043959,0.005706,0.034911,-0.006187,-0.001528,-0.006232,...,-0.017560,0.005680,0.117681,0.037063,0.032127,0.0,3.965082e-18,0.083172,0.012056,0.025169
2020-08-01 00:00:00+00:00,0.0,-0.005620,-0.019859,0.008437,0.036100,0.002482,0.038633,-0.016482,0.002008,-0.009768,...,-0.016422,0.012753,0.126823,0.025853,0.016780,0.0,3.965082e-18,0.072703,0.008037,0.017888
2020-08-02 00:00:00+00:00,0.0,-0.014332,-0.021880,0.003803,0.026629,-0.004685,0.036973,-0.026203,-0.001528,-0.011036,...,-0.003744,0.023363,0.125599,0.013175,0.010244,0.0,3.965082e-18,0.063008,0.004822,0.010163


In [23]:
# Get processed CasesCapita DataFrame
df_CasesCapita = pandas.read_csv(os.path.join(data_subdirectory, 'df_CasesCapita.csv'), index_col='timestamp')
df_CasesCapita.index = pandas.to_datetime(df_CasesCapita.index)
df_CasesCapita.columns = df_CasesCapita.columns.astype(int)
df_CasesCapita.columns.name = 'pairs_id'
df_CasesCapita.tail()

pairs_id,50001,50002,50003,50004,50005,50006,50007,50008,50009,50010,...,53132,53133,53134,53135,53136,53137,53138,53139,53140,53141
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-07-29 00:00:00+00:00,NaN,27.066837,30.002773,48.088515,28.745824,34.733372,37.827522,34.275777,23.139478,25.839482,...,7.749273,2.729575,11.375624,5.008923,12.939305,NaN,0.911114,1.107298,0.567366,5.850551
2020-07-30 00:00:00+00:00,NaN,26.956556,29.102869,48.675655,29.445062,33.976851,37.543910,33.803712,24.083157,26.375943,...,7.313160,3.013695,12.559707,5.417818,13.995585,NaN,0.867102,1.107298,0.567366,6.398214
2020-07-31 00:00:00+00:00,NaN,25.115464,27.460577,44.560099,28.862407,31.783366,38.434965,30.729202,23.881394,29.062862,...,7.997736,3.013695,13.867041,5.417818,13.710928,NaN,0.825217,1.107298,0.567366,6.337479
2020-08-01 00:00:00+00:00,NaN,24.877053,27.099816,44.324847,28.291281,31.733508,40.832930,28.888046,23.881394,30.537996,...,8.538693,3.013695,14.999055,5.008923,12.314274,NaN,0.762936,1.190270,0.567366,6.420551
2020-08-02 00:00:00+00:00,NaN,23.226029,26.923148,43.296891,27.357261,29.889059,43.151480,27.664014,22.727807,31.115683,...,9.704510,3.013695,15.947930,4.698435,11.857741,NaN,0.705355,1.279459,0.567366,6.626436


In [24]:
# Get processed Mobility DataFrame
df_Mobility = pandas.read_csv(os.path.join(data_subdirectory, 'df_Mobility.csv'), index_col='timestamp')
df_Mobility.index = pandas.to_datetime(df_Mobility.index)
df_Mobility.columns = df_Mobility.columns.astype(int)
df_Mobility.columns.name = 'pairs_id'
df_Mobility.tail()

pairs_id,50001,50002,50003,50004,50005,50006,50007,50008,50009,50010,...,53132,53133,53134,53135,53136,53137,53138,53139,53140,53141
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-07-29 00:00:00+00:00,8.874857,10.466500,7.389429,7.701857,14.377643,8.945214,10.908143,11.178786,10.404857,9.866929,...,9.709929,4.135357,57.702357,3.952214,4.511000,NaN,5.277500,2.709357,6.745929,4.533429
2020-07-30 00:00:00+00:00,8.841857,10.357143,7.313143,7.705500,14.482286,8.899286,10.864929,11.097286,10.193214,9.863214,...,9.604357,4.121857,57.437571,3.930643,4.487286,NaN,5.183929,2.617429,6.824571,4.525857
2020-07-31 00:00:00+00:00,8.788071,10.405357,7.236071,7.711143,14.571500,8.739286,11.043286,11.187500,10.159000,9.875286,...,9.637643,4.136357,58.462429,4.010429,4.500286,NaN,5.370500,2.443357,6.890286,4.534786
2020-08-01 00:00:00+00:00,8.773714,10.348571,7.204714,7.683143,14.604857,8.705643,11.101000,11.269357,10.078357,9.747071,...,9.452214,4.124500,56.027500,4.080357,4.476429,NaN,4.743786,2.363071,6.844000,4.529071
2020-08-02 00:00:00+00:00,8.720500,10.299571,7.151643,7.718143,14.677714,8.704143,11.017357,11.192357,10.200357,9.653000,...,9.158929,4.060143,55.206000,4.094429,4.394571,NaN,4.664929,2.315000,6.555857,4.467929


In [25]:
# Get processed LogMobility DataFrame
df_LogMobility = pandas.read_csv(os.path.join(data_subdirectory, 'df_LogMobility.csv'), index_col='timestamp')
df_LogMobility.index = pandas.to_datetime(df_LogMobility.index)
df_LogMobility.columns = df_LogMobility.columns.astype(int)
df_LogMobility.columns.name = 'pairs_id'
df_LogMobility.tail()

pairs_id,50001,50002,50003,50004,50005,50006,50007,50008,50009,50010,...,53132,53133,53134,53135,53136,53137,53138,53139,53140,53141
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-07-29 00:00:00+00:00,2.183222,2.348180,2.000050,2.041461,2.665674,2.191119,2.389510,2.414018,2.342273,2.289189,...,2.273149,1.419574,4.055298,1.374276,1.506519,NaN,1.663453,0.996711,1.908939,1.511479
2020-07-30 00:00:00+00:00,2.179497,2.337676,1.989673,2.041934,2.672926,2.185971,2.385540,2.406701,2.321722,2.288812,...,2.262217,1.416304,4.050699,1.368803,1.501248,NaN,1.645563,0.962192,1.920530,1.509807
2020-07-31 00:00:00+00:00,2.173395,2.342321,1.979078,2.042666,2.679068,2.167828,2.401823,2.414797,2.318360,2.290035,...,2.265677,1.419815,4.068384,1.388898,1.504141,NaN,1.680921,0.893373,1.930113,1.511778
2020-08-01 00:00:00+00:00,2.171760,2.336848,1.974736,2.039029,2.681354,2.163971,2.407035,2.422087,2.310390,2.276967,...,2.246249,1.416945,4.025843,1.406185,1.498826,NaN,1.556835,0.859962,1.923372,1.510517
2020-08-02 00:00:00+00:00,2.165677,2.332102,1.967342,2.043574,2.686330,2.163799,2.399472,2.415231,2.322423,2.267269,...,2.214729,1.401218,4.011072,1.409627,1.480370,NaN,1.540073,0.839410,1.880359,1.496925


In [26]:
#One-time calculation
"""
# Pairwise distance between county centroids

def haversine(lat1,lon1,lat2,lon2):
    #This uses the ‘haversine’ formula to calculate the great-circle distance between two points – that is, 
    #the shortest distance over the earth’s surface – giving an ‘as-the-crow-flies’ distance between the points 
    #(ignoring any hills they fly over, of course!).
    #Haversine
    #formula:    a = sin²(Δφ/2) + cos φ1 ⋅ cos φ2 ⋅ sin²(Δλ/2)
    #c = 2 ⋅ atan2( √a, √(1−a) )
    #d = R ⋅ c
    #where   φ is latitude, λ is longitude, R is earth’s radius (mean radius = 6,371km);
    #note that angles need to be in radians to pass to trig functions!
    R = 6371.0088 #km
    lat1,lon1,lat2,lon2 = map(numpy.radians, [lat1,lon1,lat2,lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = numpy.sin(dlat/2)**2 + numpy.cos(lat1) * numpy.cos(lat2) * numpy.sin(dlon/2) **2
    c = 2 * numpy.arctan2(a**0.5, (1-a)**0.5)
    d = R * c
    return round(d,4)

haversine_vec = numpy.vectorize(haversine)

# Get the polygon centroids
df_region['centroid'] = df_region['poly'].apply(lambda x: x.centroid)

# Extract the lats and lons of the centroids
df_centroids = df_region[['pairs_id', 'centroid']].set_index('pairs_id')
lons = df_centroids['centroid'].apply(lambda x: x.coords.xy[0][0]).values
lats = df_centroids['centroid'].apply(lambda x: x.coords.xy[1][0]).values
df_centroids.tail()

# Empty array to hold the results of pointwise Haversine
distance = numpy.zeros((len(lons), len(lons)))
distance[:] = numpy.nan

# Haversine distance
for i, (lon1, lat1) in enumerate(zip(lons, lats)):
    distance[i, :i+1] = haversine_vec(lats[:i+1], lons[:i+1], lat1, lon1)
    
# Numpy to Pandas and filling in nan
df_distance = pandas.DataFrame(distance)
df_distance = df_distance.fillna(pandas.DataFrame(distance.T)) # Since we only calculated half the values
df_distance.columns = list(df_centroids.index)
df_distance.index = list(df_centroids.index)

# Write to disk
df_distance.to_csv('data/df_distance.csv')
"""

"\n# Pairwise distance between county centroids\n\ndef haversine(lat1,lon1,lat2,lon2):\n    #This uses the ‘haversine’ formula to calculate the great-circle distance between two points – that is, \n    #the shortest distance over the earth’s surface – giving an ‘as-the-crow-flies’ distance between the points \n    #(ignoring any hills they fly over, of course!).\n    #Haversine\n    #formula:    a = sin²(Δφ/2) + cos φ1 ⋅ cos φ2 ⋅ sin²(Δλ/2)\n    #c = 2 ⋅ atan2( √a, √(1−a) )\n    #d = R ⋅ c\n    #where   φ is latitude, λ is longitude, R is earth’s radius (mean radius = 6,371km);\n    #note that angles need to be in radians to pass to trig functions!\n    R = 6371.0088 #km\n    lat1,lon1,lat2,lon2 = map(numpy.radians, [lat1,lon1,lat2,lon2])\n\n    dlat = lat2 - lat1\n    dlon = lon2 - lon1\n    a = numpy.sin(dlat/2)**2 + numpy.cos(lat1) * numpy.cos(lat2) * numpy.sin(dlon/2) **2\n    c = 2 * numpy.arctan2(a**0.5, (1-a)**0.5)\n    d = R * c\n    return round(d,4)\n\nhaversine_vec = numpy.v

In [27]:
# Read county-to-county distance matrix from disk
df_distance = pandas.read_csv('data/df_distance.csv', index_col=0)

# Make a copy in the specific subfolder
df_distance.to_csv(os.path.join(data_subdirectory, 'df_distance.csv'))

# Read back from disk
df_distance = pandas.read_csv('data/df_distance.csv', index_col=0)
df_distance.columns = list(df_distance.columns.astype(int))

df_distance.tail()

,50001,50002,50003,50004,50005,50006,50007,50008,50009,50010,...,53132,53133,53134,53135,53136,53137,53138,53139,53140,53141
53137,1709.1757,1764.0542,1799.2760,1711.8327,1759.5311,1797.4661,1726.6741,1744.9659,1780.3946,1838.8778,...,537.8090,188.8930,252.2302,186.6679,394.1078,0.0000,84.3556,108.1013,110.2552,195.5690
53138,1780.9233,1836.4976,1872.1593,1782.8840,1830.8856,1869.2925,1797.0662,1814.5783,1850.8175,1912.4664,...,456.1365,104.8634,184.0870,147.6388,313.7357,84.3556,0.0000,136.4658,103.3369,197.3083
53139,1654.5569,1710.9309,1747.1261,1655.7203,1704.0235,1742.9476,1669.1834,1685.8989,1722.9272,1788.3516,...,522.4967,222.7925,218.0869,123.5949,378.3107,108.1013,136.4658,0.0000,50.9293,91.5573
53140,1703.1532,1759.7490,1796.0823,1704.0643,1752.4568,1791.5351,1717.2767,1733.6882,1771.0079,1837.5310,...,473.1097,176.2099,169.8535,82.3576,328.5583,110.2552,103.3369,50.9293,0.0000,94.9014
53141,1648.2945,1705.9803,1743.0285,1647.9589,1696.7228,1736.5218,1659.9883,1675.0468,1713.5972,1785.7241,...,489.8743,253.5530,191.8369,94.6943,351.6616,195.5690,197.3083,91.5573,94.9014,0.0000


In [28]:
# Diffusion Term: Model the spread of covid19 from highly infected counties to other counties

fill_value = -4
# Log new cases
df_log_cases = df_LogCases.fillna(fill_value)
logCases = df_log_cases.values

# Not all pairs_ids are actually used in cases only these: df_log_cases.columns.values
df_distance_used = df_distance[df_log_cases.columns.values].T[df_log_cases.columns.values]
distance_used = df_distance_used.values
distance_used[distance_used==0] = numpy.nan # avoid division by zero for diagonal elements

# Clip to avoid outliers through division by small number
distance_used = numpy.clip(distance_used, a_min=10, a_max=None)

# Population (total for county)
df_pop = df_population.set_index('pairs_id').T[df_log_cases.columns.values]
pop = df_pop.values[0]

# Use 3D array for speedy calculation. Dimension 0  is time
logCases1 = logCases[:, :, None]  # other counties
pop1 = pop[None, :, None]   # other counties

#casesCapita0 = cases_capita[:, None, :]  # own county
logCases0 = logCases[:, None, :]  # own county
logCases_null = logCases0.copy()  # own county
logCases_null[:] = fill_value  # this makes sure the diagonal elements are nan

weighted_difference = ((logCases1 - logCases0) * pop1 / 1e5
                       / distance_used[None, :, :])

# Sum up the weighted differences
summed = numpy.nansum(weighted_difference, axis=1)  # sum over other counties

df_Diffusion = df_log_cases.copy() #Just to get the indices and columns right
df_Diffusion[:] = summed

# Write to disk
df_Diffusion.to_csv(os.path.join(data_subdirectory, 'df_Diffusion.csv'))

df_Diffusion.tail()

pairs_id,50001,50002,50003,50004,50005,50006,50007,50008,50009,50010,...,53132,53133,53134,53135,53136,53137,53138,53139,53140,53141
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-07-29 00:00:00+00:00,29.233896,0.905330,-4.929602,0.651715,4.971770,-0.755346,3.913704,4.549411,7.171501,3.975808,...,7.495659,6.120504,7.076455,6.892483,4.490317,16.556231,12.277841,12.649599,15.080061,4.547168
2020-07-30 00:00:00+00:00,29.249426,0.935051,-4.788451,0.620620,4.891454,-0.651237,3.958085,4.616219,7.030161,3.908962,...,7.615864,5.911952,6.855667,6.711259,4.318083,16.557050,12.384538,12.649592,15.079821,4.329344
2020-07-31 00:00:00+00:00,29.268291,1.237544,-4.533095,0.981039,4.984420,-0.367581,3.883936,4.990358,7.076084,3.516766,...,7.419323,5.918840,6.643259,6.718202,4.366413,16.565582,12.498556,12.658454,15.087774,4.359753
2020-08-01 00:00:00+00:00,29.267407,1.273344,-4.481309,1.000426,5.059674,-0.366469,3.649897,5.219633,7.069910,3.304914,...,7.270232,5.922198,6.473579,6.903075,4.596970,16.571049,12.671560,12.500384,15.093584,4.336547
2020-08-02 00:00:00+00:00,29.240776,1.522583,-4.485242,1.065914,5.163589,-0.153243,3.412917,5.357761,7.235210,3.198360,...,6.968917,5.912718,6.328416,7.041663,4.664972,16.563855,12.831868,12.328923,15.086337,4.255394


In [29]:
# Get processed Diffusion DataFrame
df_Diffusion = pandas.read_csv(os.path.join(data_subdirectory, 'df_Diffusion.csv'), index_col='timestamp')
df_Diffusion.index = pandas.to_datetime(df_Diffusion.index)
df_Diffusion.columns = df_Diffusion.columns.astype(int)
df_Diffusion.columns.name = 'pairs_id'
df_Diffusion.tail()

pairs_id,50001,50002,50003,50004,50005,50006,50007,50008,50009,50010,...,53132,53133,53134,53135,53136,53137,53138,53139,53140,53141
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-07-29 00:00:00+00:00,29.233896,0.905330,-4.929602,0.651715,4.971770,-0.755346,3.913704,4.549411,7.171501,3.975808,...,7.495659,6.120504,7.076455,6.892483,4.490317,16.556231,12.277841,12.649599,15.080061,4.547168
2020-07-30 00:00:00+00:00,29.249426,0.935051,-4.788451,0.620620,4.891454,-0.651237,3.958085,4.616219,7.030161,3.908962,...,7.615864,5.911952,6.855667,6.711259,4.318083,16.557050,12.384538,12.649592,15.079821,4.329344
2020-07-31 00:00:00+00:00,29.268291,1.237544,-4.533095,0.981039,4.984420,-0.367581,3.883936,4.990358,7.076084,3.516766,...,7.419323,5.918840,6.643259,6.718202,4.366413,16.565582,12.498556,12.658454,15.087774,4.359753
2020-08-01 00:00:00+00:00,29.267407,1.273344,-4.481309,1.000426,5.059674,-0.366469,3.649897,5.219633,7.069910,3.304914,...,7.270232,5.922198,6.473579,6.903075,4.596970,16.571049,12.671560,12.500384,15.093584,4.336547
2020-08-02 00:00:00+00:00,29.240776,1.522583,-4.485242,1.065914,5.163589,-0.153243,3.412917,5.357761,7.235210,3.198360,...,6.968917,5.912718,6.328416,7.041663,4.664972,16.563855,12.831868,12.328923,15.086337,4.255394


In [30]:
# df0: Mobility
df0 = df_Mobility.bfill().ffill()
# Create the multiindex
df0.columns = pandas.MultiIndex.from_product([df0.columns, ['Mobility']])
df0.columns.names = ['pairs_id', None]

# df1: LogMobility
df1 = df_LogMobility.bfill().ffill()
# Create the multiindex
df1.columns = pandas.MultiIndex.from_product([df1.columns, ['LogMobility']])
df1.columns.names = ['pairs_id', None]

# df2: GrowthCases
# Note: no .bfill().ffill() for GrowthCases. Instead fillna(0) before taking rolling mean
df2 = df_GrowthCases
# Create the multiindex
df2.columns = pandas.MultiIndex.from_product([df2.columns, ['GrowthCases']])
df2.columns.names = ['pairs_id', None]

# FEATURES
# df3: LogCases
df3 = df_LogCases.bfill().ffill()
# Create the multiindex
df3.columns = pandas.MultiIndex.from_product([df3.columns, ['LogCases']])
df3.columns.names = ['pairs_id', None]

# df4: CasesCapita
df4 = df_CasesCapita.bfill().ffill()
# Create the multiindex
df4.columns = pandas.MultiIndex.from_product([df4.columns, ['CasesCapita']])
df4.columns.names = ['pairs_id', None]

# df5: Diffusion
df5 = df_Diffusion.bfill().ffill()
# Create the multiindex
df5.columns = pandas.MultiIndex.from_product([df5.columns, ['Diffusion']])
df5.columns.names = ['pairs_id', None]

# CumCasesCapita
df_CumCasesCapita = df_CasesCapita.bfill().ffill().cumsum()
# Create the multiindex
df_CumCasesCapita.columns = pandas.MultiIndex.from_product([df_CumCasesCapita.columns, ['CumCasesCapita']])
df_CumCasesCapita.columns.names = ['pairs_id', None]

# Maximum mobililty between the data started and start of training
df_MaxMobility = df_Mobility.copy()
df_MaxMobility.loc[df_MaxMobility.index>dt_cutoff_training_COVID] = numpy.nan
df_MaxMobility = df_MaxMobility.groupby([True]*len(df_MaxMobility)).transform('max')
# Create the multiindex
df_MaxMobility.columns = pandas.MultiIndex.from_product([df_MaxMobility.columns, ['MaxMobility']])
df_MaxMobility.columns.names = ['pairs_id', None]

df_joined = df0.join(df1)
df_joined = df_joined.join(df2)
df_joined = df_joined.join(df3)
df_joined = df_joined.join(df4)
df_joined = df_joined.join(df5)
df_joined = df_joined.join(df_CumCasesCapita)
df_joined = df_joined.join(df_MaxMobility)

df_joined = df_joined.swaplevel(axis=1)

# Write to disk
df_joined.to_csv(os.path.join(data_subdirectory, 'df_joined.csv'))

df_joined.tail()

Mobility                                            \
pairs_id                      50001      50002     50003     50004      50005   
timestamp                                                                       
2020-07-29 00:00:00+00:00  8.874857  10.466500  7.389429  7.701857  14.377643   
2020-07-30 00:00:00+00:00  8.841857  10.357143  7.313143  7.705500  14.482286   
2020-07-31 00:00:00+00:00  8.788071  10.405357  7.236071  7.711143  14.571500   
2020-08-01 00:00:00+00:00  8.773714  10.348571  7.204714  7.683143  14.604857   
2020-08-02 00:00:00+00:00  8.720500  10.299571  7.151643  7.718143  14.677714   

                                                                      \
pairs_id                      50006      50007      50008      50009   
timestamp                                                              
2020-07-29 00:00:00+00:00  8.945214  10.908143  11.178786  10.404857   
2020-07-30 00:00:00+00:00  8.899286  10.864929  11.097286  10.193214   
2020-07-31 00:00:00+00:00  8.739286  11.043286  11.187500  10.159000   
2020-08-01 00:00:00+00:00  8.705643  11.101000  11.269357  10.078357   
2020-08-02 00:00:00+00:00  8.704143  11.017357  11.192357  10.200357   

                                     ... MaxMobility                       \
pairs_id                      50010  ...       53132     53133      53134   
timestamp                            ...                                    
2020-07-29 00:00:00+00:00  9.866929  ...   12.504929  4.916071  71.578571   
2020-07-30 00:00:00+00:00  9.863214  ...   12.504929  4.916071  71.578571   
2020-07-31 00:00:00+00:00  9.875286  ...   12.504929  4.916071  71.578571   
2020-08-01 00:00:00+00:00  9.747071  ...   12.504929  4.916071  71.578571   
2020-08-02 00:00:00+00:00  9.653000  ...   12.504929  4.916071  71.578571   

                                                                       \
pairs_id                    53135     53136 53137     53138     53139   
timestamp                                                               
2020-07-29 00:00:00+00:00  4.7655  5.909786   NaN  7.725833  4.243571   
2020-07-30 00:00:00+00:00  4.7655  5.909786   NaN  7.725833  4.243571   
2020-07-31 00:00:00+00:00  4.7655  5.909786   NaN  7.725833  4.243571   
2020-08-01 00:00:00+00:00  4.7655  5.909786   NaN  7.725833  4.243571   
2020-08-02 00:00:00+00:00  4.7655  5.909786   NaN  7.725833  4.243571   

                                               
pairs_id                      53140     53141  
timestamp                                      
2020-07-29 00:00:00+00:00  9.998929  4.868714  
2020-07-30 00:00:00+00:00  9.998929  4.868714  
2020-07-31 00:00:00+00:00  9.998929  4.868714  
2020-08-01 00:00:00+00:00  9.998929  4.868714  
2020-08-02 00:00:00+00:00  9.998929  4.868714  

[5 rows x 25088 columns]

In [31]:
# Get processed and joined DataFrame
df_joined = pandas.read_csv(os.path.join(data_subdirectory, 'df_joined.csv'), header=[0,1], index_col=0)
df_joined.index = pandas.to_datetime(df_joined.index)
# Change type of pairs_id in multiindex column to integer by transposing twice
df_joined = df_joined.T.reset_index()
df_joined['pairs_id'] = df_joined['pairs_id'].astype(int)
df_joined = df_joined.set_index(['level_0', 'pairs_id']).T
df_joined.columns.names = [None, 'pairs_id']
# Need to make it datetime index again  dtype='datetime64[ns, UTC]'
df_joined.index = df_joined.reset_index()['timestamp']
df_joined

Mobility                                            \
pairs_id                      50001      50002     50003     50004      50005   
timestamp                                                                       
2020-03-01 00:00:00+00:00  7.224000   9.717000  7.623000  7.120000  11.711000   
2020-03-02 00:00:00+00:00  8.231000  12.095000  8.757500  7.632000  15.214000   
2020-03-03 00:00:00+00:00  8.759000  12.958333  9.251667  8.020667  16.165000   
2020-03-04 00:00:00+00:00  9.037750  13.501500  9.522500  8.125750  16.811000   
2020-03-05 00:00:00+00:00  9.338000  13.985200  9.793600  8.313200  17.352600   
...                             ...        ...       ...       ...        ...   
2020-07-29 00:00:00+00:00  8.874857  10.466500  7.389429  7.701857  14.377643   
2020-07-30 00:00:00+00:00  8.841857  10.357143  7.313143  7.705500  14.482286   
2020-07-31 00:00:00+00:00  8.788071  10.405357  7.236071  7.711143  14.571500   
2020-08-01 00:00:00+00:00  8.773714  10.348571  7.204714  7.683143  14.604857   
2020-08-02 00:00:00+00:00  8.720500  10.299571  7.151643  7.718143  14.677714   

                                                                      \
pairs_id                      50006      50007      50008      50009   
timestamp                                                              
2020-03-01 00:00:00+00:00  7.503000   7.907000   8.907000   9.964000   
2020-03-02 00:00:00+00:00  8.702500  10.145500  11.215500  11.238500   
2020-03-03 00:00:00+00:00  9.419333  11.341667  11.669667  11.665000   
2020-03-04 00:00:00+00:00  9.662500  11.864500  11.567500  11.526750   
2020-03-05 00:00:00+00:00  9.899000  12.099800  11.932200  11.651000   
...                             ...        ...        ...        ...   
2020-07-29 00:00:00+00:00  8.945214  10.908143  11.178786  10.404857   
2020-07-30 00:00:00+00:00  8.899286  10.864929  11.097286  10.193214   
2020-07-31 00:00:00+00:00  8.739286  11.043286  11.187500  10.159000   
2020-08-01 00:00:00+00:00  8.705643  11.101000  11.269357  10.078357   
2020-08-02 00:00:00+00:00  8.704143  11.017357  11.192357  10.200357   

                                     ... MaxMobility                       \
pairs_id                      50010  ...       53132     53133      53134   
timestamp                            ...                                    
2020-03-01 00:00:00+00:00  7.012000  ...   12.504929  4.916071  71.578571   
2020-03-02 00:00:00+00:00  8.254500  ...   12.504929  4.916071  71.578571   
2020-03-03 00:00:00+00:00  8.739667  ...   12.504929  4.916071  71.578571   
2020-03-04 00:00:00+00:00  9.108250  ...   12.504929  4.916071  71.578571   
2020-03-05 00:00:00+00:00  9.186000  ...   12.504929  4.916071  71.578571   
...                             ...  ...         ...       ...        ...   
2020-07-29 00:00:00+00:00  9.866929  ...   12.504929  4.916071  71.578571   
2020-07-30 00:00:00+00:00  9.863214  ...   12.504929  4.916071  71.578571   
2020-07-31 00:00:00+00:00  9.875286  ...   12.504929  4.916071  71.578571   
2020-08-01 00:00:00+00:00  9.747071  ...   12.504929  4.916071  71.578571   
2020-08-02 00:00:00+00:00  9.653000  ...   12.504929  4.916071  71.578571   

                                                                       \
pairs_id                    53135     53136 53137     53138     53139   
timestamp                                                               
2020-03-01 00:00:00+00:00  4.7655  5.909786   NaN  7.725833  4.243571   
2020-03-02 00:00:00+00:00  4.7655  5.909786   NaN  7.725833  4.243571   
2020-03-03 00:00:00+00:00  4.7655  5.909786   NaN  7.725833  4.243571   
2020-03-04 00:00:00+00:00  4.7655  5.909786   NaN  7.725833  4.243571   
2020-03-05 00:00:00+00:00  4.7655  5.909786   NaN  7.725833  4.243571   
...                           ...       ...   ...       ...       ...   
2020-07-29 00:00:00+00:00  4.7655  5.909786   NaN  7.725833  4.243571   
2020-07-30 00:00:00+00:00  4.7655  5.909786   NaN  7.725833  4.243571   
2